In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession

/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/cloudpickle.py:365: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if obj.__module__ is "__builtin__":


In [2]:
spark = SparkSession.builder.appName('LR_Proj').getOrCreate()

In [3]:
data = spark.read.csv('cruise_info.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [6]:
data.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [7]:
from pyspark.ml.feature import StringIndexer

In [8]:
for item in data.head(5):
    print(item)

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)
Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)
Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)
Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)
Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)


In [9]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_Index')
indexed = indexer.fit(data).transform(data)

In [10]:
for item in indexed.head(5):
    print(item)

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_Index=16.0)
Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_Index=16.0)
Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, Cruise_line_Index=1.0)
Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, Cruise_line_Index=1.0)
Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, Cruise_line_Index=1.0)


In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_Index']

In [31]:
assembler = VectorAssembler(
        inputCols=['Age',
                    'Tonnage',
                    'passengers',
                    'length',
                    'cabins',
                    'passenger_density',
                    'Cruise_line_Index'],
        outputCol='features'
)

In [32]:
output = assembler.transform(indexed)

In [33]:
final_data = output.select('features', 'crew')

In [34]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [35]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [36]:
from pyspark.ml.regression import LinearRegression

In [37]:
lr = LinearRegression(labelCol='crew')

In [38]:
lr_model = lr.fit(train_data)

In [39]:
test_results = lr_model.evaluate(test_data)

In [40]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -1.2182903876679259|
|  1.4650702004119713|
| 0.18822121089270105|
| 0.39728546626318817|
| -0.9836907769322334|
|  0.3479511380288365|
|  -0.803286601132184|
|  0.7409935193186143|
|  2.0670690803403042|
| -0.2747995443329021|
|-0.15519065048554026|
| -0.8346007697642115|
| -1.4249985189171621|
|  -1.416211799731073|
|-0.11598650358676799|
|-0.37167935819973597|
|  0.8180886104244358|
| 0.03173019405574529|
|  0.9007099589426826|
|  0.9007099589426826|
+--------------------+
only showing top 20 rows



In [41]:
test_results.rootMeanSquaredError

0.8982639859640155

In [42]:
test_results.r2

0.9206740975391816

In [43]:
unlabeled_data = test_data.select('features')

In [44]:
predictions = lr_model.transform(unlabeled_data)

In [45]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 9.218290387667926|
|[5.0,122.0,28.5,1...| 5.234929799588029|
|[5.0,133.5,39.59,...|  12.9417787891073|
|[6.0,113.0,37.82,...|11.602714533736812|
|[7.0,116.0,31.0,9...|12.983690776932233|
|[8.0,77.499,19.5,...| 8.652048861971164|
|[8.0,110.0,29.74,...|12.403286601132184|
|[9.0,88.5,21.24,9...| 9.559006480681386|
|[10.0,46.0,7.0,6....|2.4029309196596955|
|[10.0,81.76899999...| 8.694799544332902|
|[10.0,91.62700000...|  9.15519065048554|
|[10.0,105.0,27.2,...|11.514600769764211|
|[10.0,138.0,31.14...|13.274998518917162|
|[11.0,138.0,31.14...|13.266211799731073|
|[12.0,2.329,0.94,...| 0.715986503586768|
|[12.0,58.6,15.66,...| 7.371679358199736|
|[12.0,88.5,21.24,...| 9.471911389575563|
|[12.0,90.09,25.01...| 8.648269805944254|
|[12.0,91.0,20.32,...| 9.089290041057318|
|[12.0,91.0,20.32,...| 9.089290041057318|
+--------------------+------------